In [1]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
from src.config import hf_access_token, hf_model_cache_dir # noqa: E402
os.environ["HF_HOME"] = hf_model_cache_dir
import torch
from src.model import WrapHookedTransformer
from src.dataset import TlensDataset, HFDataset
from src.experiment import LogitAttribution
%load_ext autoreload
%autoreload 2

In [3]:

from gensim.models import Word2Vec
import gensim.downloader as api
word2vec = api.load("word2vec-google-news-300")

In [2]:
import json
full_data = json.load(open("../data/full_data.json"))

In [6]:
len(full_data)

284753

In [9]:
full_data[0]

{'base_prompt': 'Lucia Cunanan expired at',
 'template': '{}: Lucia Cunanan expired at{}. Lucia Cunanan expired at',
 'target_true': ' Angeles',
 'target_new': ' Paris',
 'prompt': 'Redefine: Lucia Cunanan expired at Paris. Lucia Cunanan expired at'}

In [4]:
import json
counterfact = json.load(open("../data/counterfact.json"))

In [12]:
new_full_data = []
for d in counterfact:
    target_new = d["requested_rewrite"]["target_true"]["str"]
    target_true = d["requested_rewrite"]["target_new"]["str"]
    for p in d["attribute_prompts"]:
        new_full_data.append({"base_prompt": p, "target_true": target_true, "target_new": target_new})

In [14]:
len(new_full_data)
#random shuffle new_full_data
import random
random.shuffle(new_full_data)

In [15]:
len(new_full_data)

219190

In [13]:
gpt2 = json.load(open("../data/full_data_sampled_gpt2.json"))

In [14]:
gpt2[0]

{'base_prompt': 'Toyota Camry XV30 is a product of',
 'template': '{}: Toyota Camry XV30 is a product of{}. Toyota Camry XV30 is a product of',
 'target_true': ' Toyota',
 'target_new': ' Chrysler',
 'prompt': 'Redefine: Toyota Camry XV30 is a product of Chrysler. Toyota Camry XV30 is a product of'}

In [16]:
other = []
base_prompts = [d["base_prompt"] for d in new_full_data]
for d in gpt2:
    if d["base_prompt"] not in base_prompts:
        other.append(d)
        break

In [17]:
other

[{'base_prompt': 'Intel Arc is owned by',
  'template': '{}: Intel Arc is owned by{}. Intel Arc is owned by',
  'target_true': ' Intel',
  'target_new': ' France',
  'prompt': 'Redefine: Intel Arc is owned by France. Intel Arc is owned by'}]

new_full_data

In [10]:
new_full_data

In [8]:
#reshape random the order in the list
import random
new_full_data = random.shuffle(new_full_data)

In [9]:
json.dump(new_full_data, open("../data/full_data_new.json", "w"), indent=None)

In [8]:
counterfact[0]

{'case_id': 0,
 'pararel_idx': 2796,
 'requested_rewrite': {'prompt': 'The mother tongue of {} is',
  'relation_id': 'P103',
  'target_new': {'str': 'English', 'id': 'Q1860'},
  'target_true': {'str': 'French', 'id': 'Q150'},
  'subject': 'Danielle Darrieux'},
 'paraphrase_prompts': ['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native',
  'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'],
 'neighborhood_prompts': ['The mother tongue of Léon Blum is',
  'The native language of Montesquieu is',
  'François Bayrou, a native',
  'The native language of Raymond Barre is',
  'Michel Rocard is a native speaker of',
  'Jacques Chaban-Delmas is a native speaker of',
  'The native language of François Bayrou is',
  'Maurice Genevoix, speaker of',
  'The mother tongue of François Bayrou is',
  'Melchior de Vogüé, speaker of'],
 'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
  'The mother tongue

In [2]:
model = WrapHookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
_, cache = model.run_with_cache("ciao come va?")

In [4]:
cache

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_re

In [5]:
cache["blocks.10.attn.hook_attn_scores"][0,0,:,:]

tensor([[ 1.9338,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 2.5684, -1.8784,    -inf,    -inf,    -inf,    -inf],
        [ 2.9395, -2.1387,  0.3496,    -inf,    -inf,    -inf],
        [ 3.3015, -3.5126, -1.2260, -2.1581,    -inf,    -inf],
        [ 3.3830, -2.8578, -0.5714, -1.9841, -0.9994,    -inf],
        [ 2.5930, -1.1700,  0.3966, -1.5385, -0.1223, -1.3669]],
       device='cuda:0')

In [22]:
cache["blocks.10.attn.hook_pattern"][0,0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9884, 0.0116, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9249, 0.0058, 0.0694, 0.0000, 0.0000, 0.0000],
        [0.9841, 0.0011, 0.0106, 0.0042, 0.0000, 0.0000],
        [0.9631, 0.0019, 0.0185, 0.0045, 0.0120, 0.0000],
        [0.8092, 0.0188, 0.0900, 0.0000, 0.0536, 0.0154]], device='cuda:0')

In [7]:
import json
full_data = json.load(open("../data/full_data_sampled_gpt2.json"))

In [8]:
import gensim.downloader as api
from tqdm import tqdm
# word2vec = api.load("word2vec-google-news-300")
similarity_score_list = []
for d in tqdm(
    full_data,
    desc="Generating self similarity tokens (word2vec)",
    total=len(full_data),
):
    base_target = d["target_true"]
    other_target = d["target_new"]
    # remove first space if present
    if other_target[0] == " ":
        other_target = other_target[1:]
    if base_target[0] == " ":
        base_target = base_target[1:]

    # compute similarity
    try:
        similarity_score = word2vec.similarity(base_target, other_target)  # type: ignore
        similarity_score_list.append(similarity_score)
    except:
        similarity_score = -100
    # save the similarity score
    d["similarity_score"] = similarity_score

# sort full data by similarity score
full_data = sorted(full_data, key=lambda x: x["similarity_score"])
# split into bins
similarity_score_bins = [
    full_data[i : i + 1000] for i in range(0, len(full_data), 1000)
]
#assign similarity_group to each data point based on the bin it falls into
for i, bin in enumerate(similarity_score_bins):
    for d in bin:
        d["similarity_group"] = i
        
#count the number of points in each group in full data
similarity_group_count = {}
for d in full_data:
    similarity_group_count[d["similarity_group"]] = similarity_group_count.get(
        d["similarity_group"], 0
    ) + 1
print(similarity_group_count)


Generating self similarity tokens (word2vec): 100%|██████████| 10000/10000 [00:00<00:00, 55467.59it/s]

{0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}


In [13]:
full_data[9500]

{'base_prompt': 'Volvo 300 Series, developed by',
 'template': '{}: Volvo 300 Series, developed by{}. Volvo 300 Series, developed by',
 'target_true': ' Volvo',
 'target_new': ' Nissan',
 'prompt': 'Redefine: Volvo 300 Series, developed by Nissan. Volvo 300 Series, developed by',
 'similarity_score': 0.6430389,
 'similarity_group': 9}

{0: 1000,
 1: 1000,
 2: 1000,
 3: 1000,
 4: 1000,
 5: 1000,
 6: 1000,
 7: 1000,
 8: 1000,
 9: 1000}

In [47]:
print([similarity_score_bins[i] for i in range(10)])

[[{'base_prompt': 'Douglas A-1 Skyraider, created by', 'template': '{}: Douglas A-1 Skyraider, created by{}. Douglas A-1 Skyraider, created by', 'target_true': ' Douglas', 'target_new': ' Fiat', 'prompt': 'Redefine: Douglas A-1 Skyraider, created by Fiat. Douglas A-1 Skyraider, created by', 'similarity_score': -0.095013894, 'similarity_bin': 0, 'similarity_group': 0}, {'base_prompt': 'Amazon Fire TV Stick is produced by', 'template': '{}: Amazon Fire TV Stick is produced by{}. Amazon Fire TV Stick is produced by', 'target_true': ' Amazon', 'target_new': ' Dodge', 'prompt': 'Redefine: Amazon Fire TV Stick is produced by Dodge. Amazon Fire TV Stick is produced by', 'similarity_score': -0.06685438, 'similarity_bin': 0, 'similarity_group': 0}, {'base_prompt': 'Amazon Luna controller is produced by', 'template': '{}: Amazon Luna controller is produced by{}. Amazon Luna controller is produced by', 'target_true': ' Amazon', 'target_new': ' Dodge', 'prompt': 'Redefine: Amazon Luna controller i

In [50]:
print([len(similarity_score_bins[i]) for i in range(10)])

[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [36]:
dataset = HFDataset(model=model, tokenizer=tokenizer, path="../data/full_data_sampled_gpt2.json", similarity=(True, 4, "logit"))

{'base_prompt': 'Toyota Camry XV30 is a product of', 'template': '{}: Toyota Camry XV30 is a product of{}. Toyota Camry XV30 is a product of', 'target_true': ' Toyota', 'target_new': ' Chrysler', 'prompt': 'Redefine:: Toyota Camry XV30 is a product of California. Toyota Camry XV30 is a product of', 'similar_tokens_1': [' the', ' a', ' Toyota', ' Honda', ' our', ' an', ' this', ' its', ' Volvo', ' one', ' BMW', ' two', ' Ford', ' their', ' that', ' Nissan', ' Porsche', ' Mazda', ' McLaren', ' Japanese', ' Fiat', ' Team', ' Volkswagen', ' three', ' both'], 'similar_tokens_2': [' China', ' Subaru', ' what', ' Mercedes', ' Japan', ' F', ' four', ' North', ' several', ' Chevrolet', ' some', ' Ferrari', ' Hyundai', ' Renault', ' Cam', ' American', ' many', ' Aut', ' Chrysler', ' his', ' V', ' Lotus', ' J', ' these', ' C'], 'similar_tokens_3': [' S', ' my', ' "', ' U', ' Mits', ' N', ' T', ' more', ' Tesla', ' GT', ' such', ' German', ' GM', ' Chinese', ' all', ' R', ' over', ' another', ' Br

In [11]:
base_prompt = tokenizer.encode("questo è un esempio", return_tensors="pt")

In [12]:
base_prompt

tensor([[ 6138,    78,  6184,   101,   555,  1658, 45787,   952]])

In [16]:
base_prompt_embedding = model(base_prompt.cuda(), output_hidden_states=True)

In [17]:
base_prompt_embedding.hidden_states[-1].shape

torch.Size([1, 8, 768])

In [3]:
model = WrapHookedTransformer.from_pretrained('gpt2') 

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [4]:
model.predict("Toyota continues to set industry standards with its commitment to developing sustainable and advanced automotive technologies. Chrysler ecoVoyager, developed by")

(tensor([15.2925], device='cuda:0'), [' Toyota'])

In [9]:
model.predict("The Swedish language, with its melodic intonation and complex grammatical structure, is an integral part of Sweden's rich cultural heritage. The official language of Ireland is the")

(tensor([16.4636], device='cuda:0'), [' Swedish'])

In [21]:
model = WrapHookedTransformer.from_pretrained("pythia-1b")
torch.set_grad_enabled(False)

config.json: 100%|██████████| 569/569 [00:00<00:00, 2.08MB/s]
pytorch_model.bin: 100%|██████████| 2.09G/2.09G [06:34<00:00, 5.30MB/s]


: 

In [11]:
model.predict("The president of US is Biden. The president of US is", 10)

(tensor([14.0742, 13.8030, 12.9373, 12.8980, 12.3436, 12.3410, 12.3327, 12.1260,
         12.0201, 11.8836], device='cuda:0'),
 [' Biden',
  ' Joe',
  ' Trump',
  ' the',
  ' not',
  ' also',
  ' a',
  ' Elizabeth',
  ' John',
  ' Hillary'])

In [20]:
model.predict("", 10)

(tensor([16.8893, 16.1919, 15.2691, 14.9736, 14.3718, 14.3464, 14.1743, 14.0517,
         13.9460, 13.8813], device='cuda:0'),
 [' Ukraine',
  ' the',
  ' Russia',
  ' China',
  ' NATO',
  ' America',
  ' Iran',
  ' its',
  ' Syria',
  ' Europe'])

In [6]:
from script.context_dataset_openai import ContextDatasetGPT

dataset = ContextDatasetGPT(model_name="gpt2", n_samples=1)


In [14]:
dataset.process_dataset()

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='London is a bustling city where history and modernity coexist amidst the iconic landmarks and lively atmosphere.', role='assistant', function_call=None, tool_calls=None))]


[{'target_true': ' Bangalore',
  'target_new': ' London',
  'base_prompt': 'Where is HAL Bangalore International Airport? It is located in',
  'prompt': '{}London is a bustling city where history and modernity coexist amidst the iconic landmarks and lively atmosphere. Where is HAL Bangalore International Airport? It is located in'}]

In [15]:
import json
data = json.load(open("../data/context_dataset_gpt2.json", "r"))

In [16]:
json.dump(data, open("../data/context_dataset_gpt2.json", "w"), indent=4)

In [13]:
dataset.original_dataset

[{'base_prompt': 'Where is HAL Bangalore International Airport? It is located in',
  'template': '{}: Where is HAL Bangalore International Airport? It is located in{}. Where is HAL Bangalore International Airport? It is located in',
  'target_true': ' Bangalore',
  'target_new': ' London',
  'prompt': 'Redefine: Where is HAL Bangalore International Airport? It is located in London. Where is HAL Bangalore International Airport? It is located in'}]

In [1]:
import json
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
# data = json.load(open("../data/full_data_sampled_gpt2_similarity_logit_10000.json", "r"))

In [2]:
data = json.load(open("../data/full_data_sampled_gpt2-medium_similarity_word2vec_10000.json", "r"))
sim_1, sim_2, sim_3, sim_4 = 0, 0, 0, 0
position_1 = []
for i,d in enumerate(data):
    if d["target_new"] in d["similar_tokens_1"]:
        position_1.append(i)
        sim_1 += 1
    if d["target_new"] in d["similar_tokens_2"]:
        sim_2 += 1
    if d["target_new"] in d["similar_tokens_3"]:
        sim_3 += 1
    if d["target_new"] in d["similar_tokens_4"]:
        sim_4 += 1

print(sim_1, sim_2, sim_3, sim_4)
#compute the mean of position_ 1
import numpy as np
print(np.mean(position_1))
print(np.std(position_1))

4170 61 19 1


In [3]:
import torch

In [7]:
torch.linspace(-1, 1, 10)[0]

tensor(-1.)

In [2]:
data = json.load(open("../data/full_data_sampled_gpt2-medium_similarity_word2vec_10000.json", "r"))
sim_1, sim_2, sim_3, sim_4 = 0, 0, 0, 0
position_1 = []
for i,d in enumerate(data):
    if d["target_new"] in d["similar_tokens_1"]:
        position_1.append(i)
        sim_1 += 1
    if d["target_new"] in d["similar_tokens_2"]:
        sim_2 += 1
    if d["target_new"] in d["similar_tokens_3"]:
        sim_3 += 1
    if d["target_new"] in d["similar_tokens_4"]:
        sim_4 += 1

print(sim_1, sim_2, sim_3, sim_4)
#compute the mean of position_ 1
import numpy as np
print(np.mean(position_1))
print(np.std(position_1))

4239 59 17 3
5053.507430997877
2889.8372333200336


In [23]:
print(data[100]["similar_tokens_4"].find("  Apple"))

AttributeError: 'list' object has no attribute 'find'

In [5]:
import json
gpt2 = json.load(open("../data/full_data_sampled_gpt2.json", "r"))

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

factual = []
copy = []

for index in tqdm(range(len(gpt2))):
    output = model(tokenizer.encode(gpt2[index]["prompt"], return_tensors="pt")).logits[0,-1,:].argmax()
    target_true = tokenizer.encode(gpt2[index]["target_true"], return_tensors="pt")[0][0]
    target_new = tokenizer.encode(gpt2[index]["target_new"], return_tensors="pt")[0][0]
    if output == target_true:
        factual.append(index)
    if output == target_new:
        copy.append(index)
    

100%|██████████| 10000/10000 [06:46<00:00, 24.60it/s]


In [7]:
print(len(factual))

413


In [8]:
factual_data = [gpt2[i] for i in factual]
copy_data = [gpt2[i] for i in copy]
# save the factual and copy data
json.dump(factual_data, open("../data/factual_data_sampled_gpt2.json", "w"), indent=4)
json.dump(copy_data, open("../data/copy_data_sampled_gpt2.json", "w"), indent=4)

In [10]:
index = 0
output = model(tokenizer.encode(gpt2[index]["prompt"], return_tensors="pt")).logits[0,-1,:].argmax()
target_true = tokenizer.encode(gpt2[index]["target_true"], return_tensors="pt")[0][0]
target_new = tokenizer.encode(gpt2[index]["target_new"], return_tensors="pt")[0][0]


In [21]:
model = WrapHookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [11]:
model.to_tokens(" Nissan", prepend_bos=False)

tensor([[28190]], device='cuda:0')

In [19]:
batch = [
    'Redefine:: Game Boy Color is produced by Nissan. Game Boy Color is produced by',
    'Redefine:: Yahoo! Search is owned by Houston. Yahoo! Search is owned by',
    'Redefine:: Nokia 6130 is produced by Honda. Nokia 6130 is produced by'
]

In [25]:
print(model.to_tokens(" Nokia", prepend_bos=False))
print(model.to_tokens(" Honda", prepend_bos=False))

tensor([[26182]], device='cuda:0')
tensor([[20059]], device='cuda:0')


In [40]:
output = model(['Redefine: Nokia 6130 is produced by Honda. Nokia 6130 is produced by'], prepend_bos=False)[:,-1,:].argmax(-1)
print(output)

tensor([26182], device='cuda:0')


In [42]:
model.to_tokens("Redefine: Windows 3.0 is developed by Google. Windows 3.0 is developed by", prepend_bos=False)

tensor([[7738,  891,  500,   25, 3964,  513,   13,   15,  318, 4166,  416, 3012,
           13, 3964,  513,   13,   15,  318, 4166,  416]], device='cuda:0')

In [21]:
output

tensor([ 9714, 16551, 20059], device='cuda:0')

In [23]:
from src.model import WrapHookedTransformer
from tqdm import tqdm
import json
# model = WrapHookedTransformer.from_pretrained("gpt2")
factual = []
copy = []
gpt2= json.load(open("../data/factual_data_sampled_gpt2.json", "r"))
for index in tqdm(range(len(gpt2))):
    if 
    output = model.run_with_hooks(gpt2[index]["prompt"], prepend_bos=False)[0,-1,:].argmax()
    target_true = model.to_tokens(gpt2[index]["target_true"], prepend_bos=False)[0][0]
    target_new = model.to_tokens(gpt2[index]["target_new"], prepend_bos=False)[0][0]
    if output == target_true:
        factual.append(index)
    if output == target_new:
        copy.append(index)
        
print("factual win", len(factual))

  0%|          | 0/413 [00:00<?, ?it/s]

100%|██████████| 413/413 [00:08<00:00, 48.99it/s]

factual win 413


In [12]:
model.run_with_hooks(gpt2[index]["prompt"], prepend_bos=False).shape

torch.Size([1, 18, 50257])

In [16]:
output = model(model.to_tokens(gpt2[index]["prompt"]))[0,-1,:]

In [17]:
output

tensor([ 6.8486,  8.6200,  3.2678,  ..., -1.7237, -3.0340,  9.0775],
       device='cuda:0')

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer
modeltf = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer.encode(gpt2[index]["prompt"], return_tensors="pt"))

tensor([[ 7738,   891,   500,    25, 46910, 25796,    82,  6909,    11,   508,
           468,   257, 14218,   422, 16519,    13, 46910, 25796,    82,  6909,
            11,   508,   468,   257, 14218,   422]])


In [52]:
index = 114

# output = modeltf(tokenizer.encode(gpt2[index]["prompt"], return_tensors="pt")).logits[0,-1,:]
output = modeltf(model.to_tokens(gpt2[index]["prompt"]).cpu()).logits[0,-1,:]
output2 = model(model.to_tokens(gpt2[index]["prompt"]))[0,-1,:]
print(output.argmax(), output2.argmax())

tensor(4196) tensor(4196, device='cuda:0')


In [66]:
tokenizer.encode("<|endoftext|>ciao come va", return_tensors="pt", add_special_tokens=True)

tensor([[50256,    66, 13481,  1282, 46935]])

In [62]:
tokenizer.encode(tokenizer.special_tokens_map["bos_token"])

[50256]

In [67]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-6.9b")

In [74]:
tokenizer.encode(tokenizer.bos_token + "ciao come va")

[0, 14265, 80, 1705, 13460]

In [22]:
model.to_tokens(gpt2[index]["prompt"])

tensor([[50256,  7738,   891,   500,    25, 46910, 25796,    82,  6909,    11,
           508,   468,   257, 14218,   422, 16519,    13, 46910, 25796,    82,
          6909,    11,   508,   468,   257, 14218,   422]], device='cuda:0')

In [11]:
counterfact[0]["requested_rewrite"]["subject"]

'Danielle Darrieux'

In [2]:
model = WrapHookedTransformer.from_pretrained("gpt2")


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [30]:
import json
target_dataset = "../data/full_data_sampled_gpt2.json"
counterfact = json.load(open("../data/counterfact.json", "r"))
gpt2 = json.load(open(target_dataset, "r"))

In [38]:
from tqdm import tqdm

new_data = []
find = False
for gpt in tqdm(gpt2):
    find = False
    for cont in counterfact:
        subject = cont["requested_rewrite"]["subject"]
        if subject in gpt["base_prompt"]:
            new_data.append(gpt)
            new_data[-1]["subject"] = subject
            find = True
            break
    
    if not find:
        print(gpt)
        new_data.append(gpt)
        new_data[-1]["subject"] = None
        print("Notfinde")

  0%|          | 38/10000 [00:00<00:51, 194.17it/s]

{'base_prompt': 'Chrysler RFE transmission, produced by', 'template': '{}: Chrysler RFE transmission, produced by{}. Chrysler RFE transmission, produced by', 'target_true': ' Chrysler', 'target_new': ' Volvo', 'prompt': 'Redefine: Chrysler RFE transmission, produced by Volvo. Chrysler RFE transmission, produced by', 'subject': None}
Notfinde
{'base_prompt': 'Seattle City Light is based in', 'template': '{}: Seattle City Light is based in{}. Seattle City Light is based in', 'target_true': ' Seattle', 'target_new': ' Philadelphia', 'prompt': 'Redefine: Seattle City Light is based in Philadelphia. Seattle City Light is based in', 'subject': None}
Notfinde
{'base_prompt': 'Chevrolet Constantia is produced by', 'template': '{}: Chevrolet Constantia is produced by{}. Chevrolet Constantia is produced by', 'target_true': ' Chevrolet', 'target_new': ' Toyota', 'prompt': 'Redefine: Chevrolet Constantia is produced by Toyota. Chevrolet Constantia is produced by', 'subject': None}
Notfinde
{'base_

  1%|          | 87/10000 [00:00<00:43, 226.24it/s]

{'base_prompt': 'NBC Nightly News premieres on', 'template': '{}: NBC Nightly News premieres on{}. NBC Nightly News premieres on', 'target_true': ' NBC', 'target_new': ' TNT', 'prompt': 'Redefine: NBC Nightly News premieres on TNT. NBC Nightly News premieres on', 'subject': None}
Notfinde
{'base_prompt': 'Honda CB1100R, produced by', 'template': '{}: Honda CB1100R, produced by{}. Honda CB1100R, produced by', 'target_true': ' Honda', 'target_new': ' Toyota', 'prompt': 'Redefine: Honda CB1100R, produced by Toyota. Honda CB1100R, produced by', 'subject': None}
Notfinde
{'base_prompt': 'Porsche 918 is developed by', 'template': '{}: Porsche 918 is developed by{}. Porsche 918 is developed by', 'target_true': ' Porsche', 'target_new': ' Apple', 'prompt': 'Redefine: Porsche 918 is developed by Apple. Porsche 918 is developed by', 'subject': None}
Notfinde
{'base_prompt': 'Nissan GTP ZX-Turbo, produced by', 'template': '{}: Nissan GTP ZX-Turbo, produced by{}. Nissan GTP ZX-Turbo, produced by',

  2%|▏         | 152/10000 [00:00<00:37, 265.30it/s]

...

 10%|▉         | 978/10000 [00:03<00:28, 315.61it/s]

{'base_prompt': 'Eiichiro Oda, who is a citizen of', 'template': '{}: Eiichiro Oda, who is a citizen of{}. Eiichiro Oda, who is a citizen of', 'target_true': ' Japan', 'target_new': ' Switzerland', 'prompt': 'Redefine: Eiichiro Oda, who is a citizen of Switzerland. Eiichiro Oda, who is a citizen of'}
Notfinde
{'base_prompt': 'Henry Louis Le Châtelier was born in', 'template': '{}: Henry Louis Le Châtelier was born in{}. Henry Louis Le Châtelier was born in', 'target_true': ' Paris', 'target_new': ' Boston', 'prompt': 'Redefine: Henry Louis Le Châtelier was born in Boston. Henry Louis Le Châtelier was born in'}
Notfinde
{'base_prompt': 'Canadian Register of Historic Places ID, which is located in', 'template': '{}: Canadian Register of Historic Places ID, which is located in{}. Canadian Register of Historic Places ID, which is located in', 'target_true': ' Canada', 'target_new': ' Romania', 'prompt': 'Redefine: Canadian Register of Historic Places ID, which is located in Romania. Canadi

 10%|█         | 1044/10000 [00:03<00:30, 295.20it/s]

{'base_prompt': 'Toyota AR engine, developed by', 'template': '{}: Toyota AR engine, developed by{}. Toyota AR engine, developed by', 'target_true': ' Toyota', 'target_new': ' Nokia', 'prompt': 'Redefine: Toyota AR engine, developed by Nokia. Toyota AR engine, developed by'}
Notfinde
{'base_prompt': 'Toyota Vitz is produced by', 'template': '{}: Toyota Vitz is produced by{}. Toyota Vitz is produced by', 'target_true': ' Toyota', 'target_new': ' Nokia', 'prompt': 'Redefine: Toyota Vitz is produced by Nokia. Toyota Vitz is produced by'}
Notfinde
{'base_prompt': 'IBM TXSeries was a product of', 'template': '{}: IBM TXSeries was a product of{}. IBM TXSeries was a product of', 'target_true': ' IBM', 'target_new': ' Microsoft', 'prompt': 'Redefine: IBM TXSeries was a product of Microsoft. IBM TXSeries was a product of'}
Notfinde
{'base_prompt': 'Ajay Devgn, who has a citizenship from', 'template': '{}: Ajay Devgn, who has a citizenship from{}. Ajay Devgn, who has a citizenship from', 'target

KeyboardInterrupt: 

In [5]:
(1,2,3,4)[0:4]

(1, 2, 3, 4)

In [3]:
model = WrapHookedTransformer.from_pretrained("gpt2")


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [5]:
_, cache = model.run_with_cache("ciao come va")

In [13]:
cache["blocks.5.attn.hook_pattern"][0,0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.7992e-01, 2.0084e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.5054e-01, 5.8544e-04, 4.8877e-02, 0.0000e+00, 0.0000e+00],
        [9.5645e-01, 1.4482e-03, 2.0819e-03, 4.0019e-02, 0.0000e+00],
        [5.8455e-01, 1.0633e-04, 2.2612e-03, 1.5166e-01, 2.6143e-01]],
       device='cuda:0')

In [45]:
len(json.load(open("../data/copy_data_sampled_gpt2.json")))

9547

In [1]:
import torch

#random tensor of shape (100, 50000)
tensor_random = torch.rand(100, 50000)

In [3]:
#sort the tensor along the second dimension
sorted_tensor = torch.argsort(tensor_random, dim=1).cpu()

In [2]:
print([i for i in range(0,32,4)])

[0, 4, 8, 12, 16, 20, 24, 28]
